<a href="https://colab.research.google.com/github/staerkjoe/QuantRisk_Colab/blob/main/PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/classification-of-travel-purpose")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'classification-of-travel-purpose' dataset.
Path to dataset files: /kaggle/input/classification-of-travel-purpose


Link to dataset:
https://www.kaggle.com/datasets/pkdarabi/classification-of-travel-purpose

In [72]:
!pip install unidecode

In [73]:
import pandas as pd
import numpy as np
import os
import time
from unidecode import unidecode

In [74]:
for file in os.listdir(path):
    print(file)

train_data.csv


In [75]:
df = pd.read_csv(os.path.join(path, "train_data.csv"))

In [76]:
# drop columns that are unnecessary/not helpful for prediction

# List of columns to drop
drop_cols = [
    'BillID',               # Unique identifier
    'TicketID',             # Unique identifier
    'UserID',               # Sparse and semi-unique
    'BuyerMobile',          # Personal identifier
    'NationalCode',         # Personal identifier
    'HashPassportNumber_p', # Sparse and hashed
    'HashEmail',            # Sparse and hashed
    'CancelTime',           # Leaks target label
    'VehicleType'
]

# Drop columns from DataFrame
df = df.drop(columns=drop_cols)

# Optional: Confirm result
print("Remaining columns:", df.columns.tolist())




Remaining columns: ['Created', 'DepartureTime', 'ReserveStatus', 'Male', 'Price', 'CouponDiscount', 'From', 'To', 'Domestic', 'VehicleClass', 'TripReason', 'Vehicle', 'Cancel']


In [77]:
# Drop Nulla values for From and TO
df = df.dropna(subset=['From', 'To'])

# Combine 'From' and 'To' columns to create a list of all cities
all_cities = df['From'].tolist() + df['To'].tolist()

# Create a set of unique cities and convert it back to a list
unique_cities = list(set(all_cities))

# print(unique_cities)

# Mapping erstellen
city_mapping = {city: unidecode(city) for city in unique_cities}

'''
# Ausgabe als Dictionary
print("city_mapping = {")
for k, v in city_mapping.items():
    print(f"    '{k}': '{v}',")
print("}")
'''

'\n# Ausgabe als Dictionary\nprint("city_mapping = {")\nfor k, v in city_mapping.items():\n    print(f"    \'{k}\': \'{v}\',")\nprint("}")\n'

In [78]:
def normalize_city_name(name):
    if pd.isna(name):
        return None
    return (
        str(name)
        .strip()
        .replace('\u200c', '')  # remove zero-width non-joiner
        .replace(' (', '(')
        .replace(')', ')')
        .replace('‌', '')       # remove zero-width joiner
    )

def safe_unidecode(value):
    if pd.isna(value):
        return ""
    return unidecode(str(value))

df['From'] = df['From'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)
df['To'] = df['To'].apply(normalize_city_name).map(city_mapping).apply(safe_unidecode)

df['FromMapped'] = df['From'].isin(city_mapping.values())
df['ToMapped'] = df['To'].isin(city_mapping.values())

# keep only rows where both 'From' and 'To' were mapped
df = df[df["FromMapped"] & df["ToMapped"]].copy()
# drop mapping flags
df.drop(columns=["FromMapped", "ToMapped"], inplace=True)

In [79]:
df['Price'].describe()

,Price
count,9.167200e+04
mean,3.333467e+06
std,7.696764e+06
min,-1.514000e+06
25%,1.025000e+06
50%,1.680000e+06
75%,3.150000e+06
max,3.837646e+08


In [80]:
q25 = df['Price'].quantile(0.25)
q75 = df['Price'].quantile(0.75)

df = df[(df['Price'] >= q25) & (df['Price'] <= q75)].copy()



In [81]:
# Columns Transformations

# Date FOrmat
df[['Created', 'DepartureTime']] = df[['Created', 'DepartureTime']].apply(pd.to_datetime)

# Price to €
exchange_rate = 0.00002027  # 1 IRR = 0.00002027 EUR
df['Price'] = (df['Price'] * exchange_rate).round(2)
df['CouponDiscount'] = (df['CouponDiscount'] * exchange_rate).round(2)


In [82]:
# Fearure Engineering
df['LeadTime'] = (df['DepartureTime'] - df['Created']).dt.total_seconds()
df['IsWeekend'] = (df['DepartureTime'].dt.weekday >= 5).astype(int)
df['DiscountRatio'] = np.where(df['Price'] > 0, df['CouponDiscount'] / df['Price'], 0)
df['BookingMonth'] = df['Created'].dt.month
df['DepartureMonth'] = df['DepartureTime'].dt.month
df['BookingWeekday'] = df['Created'].dt.weekday
df['DepartureWeekday'] = df['DepartureTime'].dt.weekday
df['DepartureHour'] = df['DepartureTime'].dt.hour
df['BookingHour'] = df['Created'].dt.hour
df["Male"] = df["Male"].astype(int)

# drop raw datetime columns
df.drop(columns=['Created', 'DepartureTime'], inplace=True)

In [83]:
# Rename the column
df.rename(columns={'VehicleClass': 'VIPStatus'}, inplace=True)

# Replace NaN values with 'Unknown'
df['VIPStatus'] = df['VIPStatus'].fillna('Unknown')
df["VIPStatus"] = df["VIPStatus"].astype(str)


In [84]:
df.head()

,ReserveStatus,Male,Price,CouponDiscount,From,To,Domestic,VIPStatus,TripReason,Vehicle,Cancel,LeadTime,IsWeekend,DiscountRatio,BookingMonth,DepartureMonth,BookingWeekday,DepartureWeekday,DepartureHour,BookingHour
2,3,0,40.54,0.0,shyrz,hwz,1,True,Work,Bus,0,63272.750,0,0.0,9,9,1,2,11,17
7,3,1,50.27,0.0,thrn,shyrz,1,True,Work,Bus,0,7215.157,0,0.0,7,7,3,3,16,13
8,3,1,24.73,0.0,thrn,Sfhn,1,True,Work,Bus,0,57761.133,0,0.0,10,10,6,0,2,9
9,3,1,26.35,0.0,thrn,tbryz,1,True,Int,Bus,0,312199.797,0,0.0,3,3,3,0,13,22
14,2,1,23.48,0.0,yzd,thrn,1,Unknown,Int,Train,0,1471280.147,0,0.0,9,9,1,4,20,20


In [85]:
# use of frequency encoding since there is a lot of different cities and other encodings would not make sense
# loss of interpretability though

#df['FromFreq'] = df['From'].map(df['From'].value_counts())
#df['ToFreq'] = df['To'].map(df['To'].value_counts())

# drop the city columns
#df.drop(columns=['From', 'To'], inplace=True)

In [86]:
df['DiscountRatio'].describe()

,DiscountRatio
count,45901.000000
mean,0.001288
std,0.015095
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.113043


In [87]:
df['Vehicle'].value_counts()

,count
Vehicle,
Bus,26712
Train,19001
Plane,184
InternationalPlane,4


In [88]:
# Create a copy of the cleaned DataFrame
cleaned_data = df.copy()

In [89]:
# Save as CSV
cleaned_data.to_csv(
    '/content/drive/MyDrive/Colab Notebooks/QantRisk/cleaned_data_new.csv',
    index=False
)

In [90]:
cleaned_data.head()

,ReserveStatus,Male,Price,CouponDiscount,From,To,Domestic,VIPStatus,TripReason,Vehicle,Cancel,LeadTime,IsWeekend,DiscountRatio,BookingMonth,DepartureMonth,BookingWeekday,DepartureWeekday,DepartureHour,BookingHour
2,3,0,40.54,0.0,shyrz,hwz,1,True,Work,Bus,0,63272.750,0,0.0,9,9,1,2,11,17
7,3,1,50.27,0.0,thrn,shyrz,1,True,Work,Bus,0,7215.157,0,0.0,7,7,3,3,16,13
8,3,1,24.73,0.0,thrn,Sfhn,1,True,Work,Bus,0,57761.133,0,0.0,10,10,6,0,2,9
9,3,1,26.35,0.0,thrn,tbryz,1,True,Int,Bus,0,312199.797,0,0.0,3,3,3,0,13,22
14,2,1,23.48,0.0,yzd,thrn,1,Unknown,Int,Train,0,1471280.147,0,0.0,9,9,1,4,20,20


In [91]:
cleaned_data['IsWeekend'].value_counts()

,count
IsWeekend,
0,33125
1,12776


In [92]:
cleaned_data['Cancel'].value_counts()

,count
Cancel,
0,38291
1,7610


In [93]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45901 entries, 2 to 101010
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ReserveStatus     45901 non-null  int64  
 1   Male              45901 non-null  int64  
 2   Price             45901 non-null  float64
 3   CouponDiscount    45901 non-null  float64
 4   From              45901 non-null  object 
 5   To                45901 non-null  object 
 6   Domestic          45901 non-null  int64  
 7   VIPStatus         45901 non-null  object 
 8   TripReason        45901 non-null  object 
 9   Vehicle           45901 non-null  object 
 10  Cancel            45901 non-null  int64  
 11  LeadTime          45901 non-null  float64
 12  IsWeekend         45901 non-null  int64  
 13  DiscountRatio     45901 non-null  float64
 14  BookingMonth      45901 non-null  int32  
 15  DepartureMonth    45901 non-null  int32  
 16  BookingWeekday    45901 non-null  int32  
 1